# MathStatistisc, lab 1-4

In [ ]:
import numpy as np
import seaborn as sns
import scipy as sp
from  matplotlib import pyplot as plt
from math import sqrt, gamma
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
dists = {'Normal': {
            'gen_data': np.random.standard_normal, 
            'cdf': sp.stats.norm.cdf, 
            'pdf': sp.stats.norm.pdf
        },
         'Cauchy': {
             'gen_data': np.random.standard_cauchy, 
             'cdf': sp.stats.cauchy.cdf, 
             'pdf':sp.stats.cauchy.pdf
         },
         'Laplace': {
             'gen_data': lambda n: np.random.laplace(0, 1 / sqrt(2), n),
             'cdf': lambda x: sp.stats.laplace.cdf(x, 0, 1 / sqrt(2)),
             'pdf': lambda x: sp.stats.laplace.pdf(x, 0, 1 / sqrt(2))
         },
         'Poisson': {
             'gen_data': lambda n: np.random.poisson(10, n), 
             'cdf': lambda x: sp.stats.poisson.cdf(x, 10), 
             'pdf': lambda x: 10 ** x * np.exp(-10) / gamma(x + 1)
         },
         'Uniform': {
             'gen_data': lambda n: np.random.uniform(-sqrt(3), sqrt(3), n), 
             'cdf': lambda x: sp.stats.uniform.cdf(x, -sqrt(3), 2 * sqrt(3)), 
             'pdf': lambda x: sp.stats.uniform.pdf(x, -sqrt(3), 2 * sqrt(3))
        }}

## lab 1

In [ ]:
def build_hist(datasets, pdf, name):
    fig, axes = plt.subplots(1, len(datasets), figsize=(20, 4))
    fig.suptitle(name)
    for i, data in enumerate(datasets):
        sns.histplot(data, kde=False, stat='density', label='samples', ax=axes[i])
        x0, x1 = axes[i].get_xlim()
        x_pdf = np.linspace(x0, x1, 100)
        y_pdf = [pdf(x) for x in x_pdf]

        axes[i].plot(x_pdf, y_pdf, 'r', lw=2, label='pdf') 
        axes[i].set_title(f'n = {len(data)}')

In [ ]:
for name, dist in dists.items():
    build_hist([dist['gen_data'](n) for n in [10, 50, 1000]], dist['pdf'], name)

## lab 2

In [ ]:
def calc_chars(data_generator, n):
    iters = 1000
    mean = 0
    med = 0
    z_r = 0
    z_q = 0
    z_tr = 0
    mean_2 = 0
    med_2 = 0
    z_r_2 = 0
    z_q_2 = 0
    z_tr_2 = 0
    for i in range(iters):
        data = data_generator(n)
        data.sort()
        
        tmp = data.mean()
        mean += tmp
        mean_2 += tmp ** 2
        
        tmp = np.median(data)
        med += tmp
        med_2 += tmp ** 2 
        
        tmp = (data[0] + data[-1]) / 2
        z_r += tmp
        z_r_2 += tmp ** 2
        
        tmp = (np.quantile(data, 0.25) + np.quantile(data, 0.75)) / 2
        z_q += tmp
        z_q_2 += tmp ** 2
        
        r = n // 4
        tmp = sum(data[r:-r]) / (n - 2 * r)
        z_tr += tmp
        z_tr_2 += tmp ** 2

    mean /= iters
    med /= iters
    z_r /= iters
    z_q /= iters
    z_tr /= iters
    mean_2 /= iters
    med_2 /= iters
    z_r_2 /= iters
    z_q_2 /= iters
    z_tr_2 /= iters

    d_mean = mean_2 - mean ** 2
    d_med = med_2 - med ** 2
    d_z_r = z_r_2 - z_r ** 2
    d_z_q = z_q_2 - z_q ** 2
    d_z_tr = z_tr_2 - z_tr ** 2

    mean= round(mean, 4)
    med= round(med, 4)
    z_r= round(z_r, 4)
    z_q= round(z_q, 4)
    z_tr= round(z_tr, 4)
    d_mean= round(d_mean, 4)
    d_med= round(d_med, 4)
    d_z_r= round(d_z_r, 4)
    d_z_q= round(d_z_q, 4)
    d_z_tr= round(d_z_tr, 4)
    
    print(f"& {mean} & {med} & {z_r} & {z_q} & {z_tr} ")
    print(f"& {d_mean} & {d_med} & {d_z_r} & {d_z_q} & {d_z_tr} ")
    print(f"& {mean - sqrt(d_mean)} & {med - sqrt(d_med)} & {z_r - sqrt(d_z_r)} & {z_q - sqrt(d_z_q)} & {z_tr - sqrt(d_z_tr)} ")
    print(f"& {mean + sqrt(d_mean)} & {med + sqrt(d_med)} & {z_r + sqrt(d_z_r)} & {z_q + sqrt(d_z_q)} & {z_tr + sqrt(d_z_tr)} ")

    mean= round(mean, 1)
    med= round(med, 1)
    z_r= round(z_r, 1)
    z_q= round(z_q, 1)
    z_tr= round(z_tr, 1)
    d_mean= round(sqrt(d_mean), 1)
    d_med= round(sqrt(d_med), 1)
    d_z_r= round(sqrt(d_z_r), 1)
    d_z_q= round(sqrt(d_z_q), 1)
    d_z_tr= round(sqrt(d_z_tr), 1)
    print(f"& ${mean}\pm{d_mean}$ & ${med}\pm{d_med}$ & ${z_r}\pm{d_z_r}$ & ${z_q}\pm{d_z_q}$ & ${z_tr}\pm{d_z_tr}& ")
    

In [ ]:
for name, dist in dists.items():
    for n in [10, 100, 1000]:
        print(f'{name}, n = {n}:')
        calc_chars(dist['gen_data'], n)
    print()

## lab 3

### Boxplots

In [ ]:
def build_boxplot(dataset, title):
    names=[str(len(data)) for data in dataset]
    fig, ax = plt.subplots(1, 1, figsize=(10, 4))
    sns.boxplot(data=dataset, orient='h', ax=ax)
    ax.set(xlabel='x', ylabel='n')
    ax.set(yticklabels=names)
    ax.set_title(title)

In [ ]:
for name, dist in dists.items():
    build_boxplot([dist['gen_data'](100), dist['gen_data'](20)], name)

### Share of outliers

In [ ]:
def calc_outlier(data_generator, n, iters=1000):
    num = 0
    for i in range(iters):
        data = data_generator(n)
        q1 = np.quantile(data, 0.25)
        q3 = np.quantile(data, 0.75)
        iqr = q3 - q1
        x1 = q1 - 1.5 * iqr
        x2 = q3 + 1.5 * iqr
        num += np.count_nonzero((data < x1) | (x2 < data)) / n
    return round(num / iters, 2)

In [ ]:
for name, dist in dists.items():
    for n in [20, 100]:
        print(f'{name} {n}:', calc_outlier(dist['gen_data'], n))

## lab 4

### Empirical distribution function

In [ ]:
def build_edf(name, datasets, cdf, x):
    fig, axes = plt.subplots(1, len(datasets), figsize=(12, 4))
    fig.suptitle(name)
    for i, data in enumerate(datasets):
        y1 = ECDF(data)(x)
        y2 = cdf(x)
        axes[i].plot(x, y1)
        axes[i].plot(x, y2)
        axes[i].set_title(f'n = {len(data)}')

In [ ]:
for name, dist in dists.items():
    if name != 'Poisson': 
        build_edf(name, [dist['gen_data'](n) for n in [20, 60, 100]], dist['cdf'], np.linspace(-4, 4, 100))
    else:
        build_edf(name, [dist['gen_data'](n) for n in [20, 60, 100]], dist['cdf'], np.linspace(6, 14, 100))

### KDE

In [ ]:
def build_kde(name, data, pdf, x):
    scales = [0.5, 1.0, 2.0]
    fig, ax = plt.subplots(1, len(scales), figsize=(12, 4))
    fig.suptitle(f'{name}, n = {len(data)}')
    for i, scale in enumerate(scales):
        sns.kdeplot(data, ax=ax[i], bw_method='silverman', bw_adjust=scale, label='kde')
        ax[i].set_xlim([x[0], x[-1]])
        ax[i].set_ylim([0, 1])
        ax[i].plot(x, [pdf(xk) for xk in x], label='pdf')
        # ax[i].legend()
        ax[i].set_title(f'h={str(scale)}*$h_n$')

In [ ]:
for name, dist in dists.items():
    for n in [20, 60, 100]:
        if name != 'Poisson':
            build_kde(name, dist['gen_data'](n), dist['pdf'], np.linspace(-4, 4, 100))
        else:
            build_kde(name, dist['gen_data'](n), dist['pdf'], np.linspace(6, 14, 100))